In [1]:
# !pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as gp  # import the installed package
from gurobipy import *
import pandas as pd
import numpy as np
import time
import datetime
df = pd.read_excel("OR110-1_case01.xlsx","Instance 1")
display(df.head())
J = df['Job ID'].shape[0]-1;
I = 4
split = df['Splitting Timing'].dropna();
due = df['Due Time'].dropna();
processNum = [2,2,2,2,2,1,2,2,2,2,2,1]
processType = df[['Process Type','Unnamed: 2']][1:]
processTime = df[['Processing Time','Unnamed: 5']][1:]

processType.rename(columns={'Unnamed: 2':'Process 2','Process Type':'Process 1'},inplace = True)
processTime.rename(columns={'Unnamed: 5':'Process 2','Processing Time':'Process 1'},inplace = True)
processTime = processTime.transpose()
display(processTime)



for i in range(due.shape[0]):
    due[i+1] = due[i+1].hour + due[i+1].minute/60 - 7.5
# print(due)

,Job ID,Process Type,Unnamed: 2,Splitting Timing,Processing Time,Unnamed: 5,Due Time
0,NaN,Process 1,Process 2,NaN,Process 1,Process 2,NaN
1,1.0,Boiling,Baking,1.0,2.7,1.3,12:30:00
2,2.0,Baking,Boiling,1.0,1.6,1.4,12:30:00
3,3.0,Boiling,Baking,1.0,0.7,1.9,12:30:00
4,4.0,Smoking,Boiling,1.0,0.5,0.7,12:30:00


,1,2,3,4,5,6,7,8,9,10,11,12
Process 1,2.7,1.6,0.7,0.5,0.8,2.5,1.4,1.1,0.8,1,3,2
Process 2,1.3,1.4,1.9,0.7,1,NaN,2,1.1,0.7,0.5,1.4,NaN


In [2]:
p1 = Model("p1")

w = []
c = []
z = []
a = []
y = []
b = []
for i in range(J):
    c.append(p1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'c' + str(i)))    
    z.append(p1.addVar(lb = 0, vtype = GRB.BINARY, name = 'z'+str(i)))

for i in range(I):
    w.append(p1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'w'+ str(i)))
             

for i in range(I):
    a.append([])
    for j in range(J):
        a[i].append(p1.addVar(lb = 0, vtype = GRB.BINARY, name = 'a' + str(i) + str(j)))
        
for p in range(J):
    y.append([])
    for q in range(J):
        if p != q:
            y[p].append(p1.addVar(lb = 0, vtype = GRB.BINARY, name = 'y' + str(p) + str(q)))

for i in range(I):
    b.append([])
    for p in range(J):
        b[i].append([])
        for q in range(J):
            if p != q:
                b[i][p].append(p1.addVar(lb = 0, vtype = GRB.BINARY, name = 'b' + str(i) + str(p) + str(q)))


p1.setObjective(quicksum(w[i] for i in range(I)) , GRB.MINIMIZE) 

# add constraints and name them
p1.addConstrs((quicksum(a[i][j] for i in range(I)) == 1 for j in range(J)), "aij")    
p1.addConstrs((w[i] >= quicksum(z[j]*a[i][j] for j in range(J) ) for i in range(I)), "wi >= zj * aij")
p1.addConstrs((c[j] >= quicksum(processTime[j+1][k] for k in range(processNum[j])) for j in range(J)), "cj")    


p1.optimize()

for var in p1.getVars():
    print(var.varName, '=', var.x)
print("objective value =", p1.objVal)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-01
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 24 rows, 736 columns and 60 nonzeros
Model fingerprint: 0xd26957fa
Model has 4 quadratic constraints
Variable types: 16 continuous, 720 integer (720 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 12 rows and 672 columns
Presolve time: 0.00s
Presolved: 64 rows, 112 columns, 244 nonzeros
Variable types: 0 continuous, 112 integer (108 binary)
Found heuristic solution: objective 0.0000000

Explored 1 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 0 

Optimal solu